In [1]:
import os 

import torch
import torch.nn as nn 
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from helpers import load_experiment

PATH_PREFIX = "/home/danis/Projects/AlphaCaption/AutoConceptBottleneck/autoconcept"

In [2]:
experiment_path = "outputs/2023-05-28/09-21-34"

dm, model = load_experiment(os.path.join(PATH_PREFIX, experiment_path))
train_loader = dm.train_dataloader()
train_set = train_loader.dataset

Global seed set to 42


Fetching configuration...
Loading datamodule...


[nltk_data] Downloading package wordnet to /home/danis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(


                             filename  \
0  Black_Footed_Albatross_0046_18.jpg   
1  Black_Footed_Albatross_0009_34.jpg   
2  Black_Footed_Albatross_0002_55.jpg   
3  Black_Footed_Albatross_0074_59.jpg   
4  Black_Footed_Albatross_0014_89.jpg   

                                     source_captions  \
0  ['closeup bin food include broccoli bread', 'm...   
1  ['giraffe eating food top tree', 'giraffe stan...   
2  ['flower vase sitting porch stand', 'white vas...   
3  ['zebra grazing lush green grass field', 'zebr...   
4  ['woman swim suit holding parasol sunny day', ...   

                                mask_source_captions  \
0  ['coco', 'coco', 'coco', 'coco', 'coco', 'cub'...   
1  ['coco', 'coco', 'coco', 'coco', 'coco', 'cub'...   
2  ['coco', 'coco', 'coco', 'coco', 'coco', 'cub'...   
3  ['coco', 'coco', 'coco', 'coco', 'coco', 'cub'...   
4  ['coco', 'coco', 'coco', 'coco', 'coco', 'cub'...   

                                          attributes  
0  [0, 0, 0, 0, 1, 0, 0,

100%|██████████| 11788/11788 [00:06<00:00, 1799.12it/s]


Max length:  373
Index for <pad>: [0]
Loading model


/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion_task' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion_task'])`.
  rank_zero_warn(
/home/danis/anaconda3/envs/bottleneck/lib/python3.10/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'criterion_tie' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['criterion_tie'])`.
  rank_zero_warn(


In [3]:
train_set[0]

{'image': tensor([[[ 1.1700,  1.1700,  1.1700,  ..., -0.4739, -0.6281, -0.6965],
          [ 1.1700,  1.1700,  1.1700,  ..., -0.3541, -0.4911, -0.5596],
          [ 1.1700,  1.1700,  1.1529,  ..., -0.1828, -0.3027, -0.3541],
          ...,
          [ 1.8722,  1.8722,  1.8722,  ...,  1.8722,  1.8722,  1.8722],
          [ 0.8961,  0.8961,  0.8961,  ...,  0.8961,  0.8961,  0.8961],
          [ 0.2453,  0.2453,  0.2453,  ...,  0.2453,  0.2453,  0.2453]],
 
         [[ 1.2206,  1.2206,  1.2206,  ..., -0.6001, -0.7577, -0.8277],
          [ 1.2206,  1.2206,  1.2206,  ..., -0.5126, -0.6527, -0.7227],
          [ 1.2206,  1.2206,  1.2031,  ..., -0.3725, -0.4951, -0.5476],
          ...,
          [ 2.0434,  2.0434,  2.0434,  ...,  2.0434,  2.0434,  2.0434],
          [ 1.0455,  1.0455,  1.0455,  ...,  1.0455,  1.0455,  1.0455],
          [ 0.3803,  0.3803,  0.3803,  ...,  0.3803,  0.3803,  0.3803]],
 
         [[ 1.7685,  1.7685,  1.7685,  ..., -0.3055, -0.4624, -0.5321],
          [ 1.7685,

In [4]:
# Shapes dataset
# [red, green, blue, square, triangle, circle, ... each for class]

attribute_mapping = {
    0: [1, 0, 0, 1, 0, 0, # 1, 0, 0, 0, 0, 0, 0, 0, 0
        ],
    1: [0, 1, 0, 1, 0, 0, # 0, 1, 0, 0, 0, 0, 0, 0, 0
        ],
    2: [0, 0, 1, 1, 0, 0, # 0, 0, 1, 0, 0, 0, 0, 0, 0
        ],
    3: [1, 0, 0, 0, 1, 0, # 0, 0, 0, 1, 0, 0, 0, 0, 0
        ],
    4: [0, 1, 0, 0, 1, 0, # 0, 0, 0, 0, 1, 0, 0, 0, 0
        ],
    5: [0, 0, 1, 0, 1, 0, # 0, 0, 0, 0, 0, 1, 0, 0, 0
        ],
    6: [1, 0, 0, 0, 0, 1, # 0, 0, 0, 0, 0, 0, 1, 0, 0
        ],
    7: [0, 1, 0, 0, 0, 1, # 0, 0, 0, 0, 0, 0, 0, 1, 0
        ],
    8: [0, 0, 1, 0, 0, 1, # 0, 0, 0, 0, 0, 0, 0, 0, 1
        ],
}

## ICLR 2018

In [5]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def prepare_data(loader, model):
    # y_train = z
    # X_train = c

    is_framework = hasattr(model.main, "concept_extractor")
    n_features = model.main.feature_extractor.main.fc.out_features

    X_train, y_train = list(), list()
    
    features_to_attributes = list()
    attribute_values = [[[0, 0] for _ in range(312)] for f in range(n_features)]
    
    for batch in tqdm(loader):
        images, targets = batch["image"].cuda(), batch["target"]
        N = images.shape[0]
        
        if is_framework:
            batch_features = model.main.inference(images)[1].cpu().detach().numpy()
        else:
            batch_features = model(images)["concept_probs"].cpu().detach().numpy()
        
        for sample_id in range(N):
            target = targets[sample_id].item()
            attributes = np.array(attribute_mapping[target])
            features = batch_features[sample_id]

            X_train.append(features)
            y_train.append(attributes)
    
    X_train, y_train = np.array(X_train), np.array(y_train)

    return X_train, y_train

X_train, y_train = prepare_data(train_loader, model)

100%|██████████| 29/29 [00:14<00:00,  1.95it/s]


In [6]:
def prepare_data(loader, model):
    # y_train = z
    # X_train = c

    is_framework = hasattr(model.main, "concept_extractor")
    n_features = model.main.feature_extractor.main.fc.out_features

    X_train, y_train = list(), list()
    
    features_to_attributes = list()
    attribute_values = [[[0, 0] for _ in range(312)] for f in range(n_features)]
    
    for batch in tqdm(loader):
        images, targets, attributes_all = batch["image"].cuda(), batch["target"], batch["attributes"]
        N = images.shape[0]
        
        if is_framework:
            batch_features = model.main.inference(images)[1].cpu().detach().numpy()
        else:
            batch_features = model(images)["concept_probs"].cpu().detach().numpy()
        
        for sample_id in range(N):
            attributes = np.array(attributes_all[sample_id])
            features = batch_features[sample_id]

            X_train.append(features)
            y_train.append(attributes)
    
    X_train, y_train = np.array(X_train), np.array(y_train)

    return X_train, y_train

X_train, y_train = prepare_data(train_loader, model)

100%|██████████| 75/75 [00:32<00:00,  2.30it/s]


In [7]:
X_train.shape, y_train[:, 0].shape

((4795, 320), (4795,))

In [8]:
# z 
# n_attributes = len(attribute_mapping[0])
n_attributes = 312

R = list()
for regressor_idx in tqdm(range(n_attributes)):
    regressor = RandomForestRegressor(random_state=42, n_estimators=20, max_depth=10)
    regressor.fit(X_train, y_train[:, regressor_idx])
    regressor.predict(X_train)
    R.append(regressor.feature_importances_)

R = np.array(R).T
R.shape

100%|██████████| 312/312 [1:16:50<00:00, 14.78s/it]


(320, 312)

In [9]:
R

array([[0.01065513, 0.00023421, 0.        , ..., 0.00150469, 0.00411365,
        0.00341284],
       [0.00104293, 0.00028857, 0.        , ..., 0.00467915, 0.00299846,
        0.00283151],
       [0.        , 0.01684333, 0.        , ..., 0.00578139, 0.00548545,
        0.00443037],
       ...,
       [0.        , 0.00907227, 0.00228571, ..., 0.008683  , 0.00115439,
        0.00370512],
       [0.        , 0.00216779, 0.        , ..., 0.00297677, 0.00657818,
        0.0022822 ],
       [0.00118685, 0.00330307, 0.00428127, ..., 0.00742537, 0.00288802,
        0.00361933]])

In [ ]:
R

array([[0.0050334 , 0.00235054, 0.00296817, ..., 0.00220805, 0.00335788,
        0.00373513],
       [0.00050124, 0.00154265, 0.00243333, ..., 0.00363363, 0.00257738,
        0.00340801],
       [0.0053127 , 0.01799806, 0.00125365, ..., 0.00480012, 0.00333179,
        0.00360335],
       ...,
       [0.00149724, 0.00840936, 0.00200709, ..., 0.00659904, 0.00258421,
        0.0026977 ],
       [0.00196808, 0.00191371, 0.00114738, ..., 0.00224508, 0.00397609,
        0.0028303 ],
       [0.00330832, 0.00170234, 0.00080454, ..., 0.0060486 , 0.00260751,
        0.00315783]])

In [9]:
TINY = 1e-12

def norm_entropy(p):
    '''p: probabilities '''
    n = p.shape[0]
    return - p.dot(np.log(p + TINY) / np.log(n + TINY))

def entropic_scores(r):
    '''r: relative importances '''
    r = np.abs(r)
    ps = r / np.sum(r, axis=0) # 'probabilities'
    hs = [1-norm_entropy(p) for p in ps.T]
    return hs

In [10]:
disent_scores = entropic_scores(R.T)
c_rel_importance = np.sum(R,1) / np.sum(R)
disent_w_avg = np.sum(np.array(disent_scores) * c_rel_importance)
print("disentanglement", disent_w_avg)

disentanglement 0.19307799240039877


In [11]:
import math

complete_scores = entropic_scores(R)
complete_scores = [v for v in complete_scores if not math.isnan(v)]
complete_avg = np.mean(complete_scores)

print("completeness", complete_avg)

completeness 0.20128058402099108


/tmp/ipykernel_26475/1556644855.py:11: RuntimeWarning: invalid value encountered in true_divide
  ps = r / np.sum(r, axis=0) # 'probabilities'


## Purity

In [ ]:
def compute_purity(loader, model):
    is_framework = hasattr(model.main, "concept_extractor")
    n_features = model.main.feature_extractor.main.fc.out_features
    
    features_to_attributes = list()
    attribute_values = [[[0, 0] for _ in range(312)] for f in range(n_features)]
    
    for batch in tqdm(loader):
        images, targets, attributes_all = batch["image"].cuda(), batch["target"], batch["attributes"]
        N = images.shape[0]
        
        if is_framework:
            batch_features = model.main.inference(images)[1].cpu().detach().numpy()
        else:
            batch_features = model(images)["concept_probs"].cpu().detach().numpy()
        
        for sample_id in range(N):
            target = targets[sample_id].item()
            attributes = np.array(attributes_all[sample_id])
            features = batch_features[sample_id]

            for feature_id in range(n_features):

                feature = features[feature_id]

                for attribute_id, attribute in enumerate(attributes):
                    value_on = attribute * feature + (1 - attribute) * (1 - feature)
                    attribute_values[feature_id][attribute_id][0] += value_on

                    value_off = attribute * (1 - feature) + (1 - attribute) * feature
                    attribute_values[feature_id][attribute_id][1] += value_off
    
    for a in attribute_values:
        a_ = [max(p) / len(train_set) for p in a]
        features_to_attributes.append(a_)
    
    return features_to_attributes

f2a = compute_purity(train_loader, model)

100%|██████████| 75/75 [2:03:49<00:00, 99.06s/it]  


In [ ]:
def compute_purity(loader, model, attribute_mapping):
    is_framework = hasattr(model.main, "concept_extractor")
    n_features = model.main.feature_extractor.main.fc.out_features
    
    features_to_attributes = list()
    attribute_values = [[[0, 0] for _ in range(len(attribute_mapping[0]))] for f in range(n_features)]
    
    for batch in tqdm(loader):
        images, targets = batch["image"].cuda(), batch["target"]
        N = images.shape[0]
        
        if is_framework:
            batch_features = model.main.inference(images)[1].cpu().detach().numpy()
        else:
            batch_features = model(images)["concept_probs"].cpu().detach().numpy()
        
        for sample_id in range(N):
            target = targets[sample_id].item()
            attributes = np.array(attribute_mapping[target])
            features = batch_features[sample_id]

            for feature_id in range(n_features):

                feature = features[feature_id]

                for attribute_id, attribute in enumerate(attributes):
                    value_on = attribute * feature + (1 - attribute) * (1 - feature)
                    attribute_values[feature_id][attribute_id][0] += value_on

                    value_off = attribute * (1 - feature) + (1 - attribute) * feature
                    attribute_values[feature_id][attribute_id][1] += value_off
    
    for a in attribute_values:
        a_ = [max(p) / len(train_set) for p in a]
        features_to_attributes.append(a_)
    
    return features_to_attributes

f2a = compute_purity(train_loader, model, attribute_mapping)

NameError: name 'attribute_mapping' is not defined

In [ ]:
def find_best_alignment(features_to_attributes, iter_converge=20.0):
    n_features, n_attributes = np.array(features_to_attributes).shape

    features_to_attributes_ = list()
    for feature_to_attributes in features_to_attributes:
        feature_to_attributes_ = sorted([(idx, fa) for idx, fa in enumerate(feature_to_attributes)], key=lambda x: x[1], reverse=True)
        features_to_attributes_.append(feature_to_attributes_)
    
    attributes_to_features = list(list() for _ in range(n_attributes))

    for idx_feat, feature_to_attributes in enumerate(features_to_attributes_):
        for idx_attr, score in feature_to_attributes:
            attributes_to_features[idx_attr].append((idx_feat, score))
    
    attributes_to_features_ = list()
    for attr2feature in attributes_to_features:
        attributes_to_features_.append(sorted(attr2feature, key=lambda x: x[1], reverse=True))
    
    best_idx = list(None for _ in range(n_features))
    best_scores = list(None for _ in range(n_features))

    patience_left = iter_converge

    while None in best_idx and patience_left > 0:
        prev_best = [_ for _ in best_idx]

        for feat_idx, f2a in enumerate(features_to_attributes_):
            
            if best_idx[feat_idx] is None:

                for att_idx, score in f2a:

                    if att_idx not in best_idx:
                        best_idx[feat_idx] = att_idx
                        best_scores[feat_idx] = score
                        break

                    else:
                        idx_other = best_idx.index(att_idx)
                        score_other = best_scores[idx_other]

                        if score > score_other:
                            best_idx[feat_idx] = att_idx
                            best_scores[feat_idx] = score

                            best_idx[idx_other] = None
                            best_scores[idx_other] = None
                            break
        
        if best_idx == prev_best:
            patience_left -= 1
        else:
            patience_left = iter_converge
        
    return list(zip(best_idx, best_scores))

    
result = find_best_alignment(f2a)

scores = [b for _, b in result if b is not None]
print("Purity: ", np.array(scores).mean())

Purity:  0.5783433018127448


## Results

### 1. Shapes dataset

| model | activation | norm_fn | slot_norm | reg_dist | f1-score | purity | disentanglement | completeness | cluster | align | directory |
|:-----------|:----:|:----:|:----:|:----:|:----:|:-------:|:-------:|:-------:|:-------:|:-------:|:-----------|
| Baseline | `sigmoid` |   `-`   |  `-`   |   `-`   | `0.830247` | `0.767724` | `0.465324 `| `0.481560` |  `A`  | `-`  | `outputs/2023-05-22/08-37-36` |
| Baseline | `gumbel` |   `-`   |   `-`   |  `-`   | `0.404321`  | `0.828083` | `0.316362` | `0.276083` | `A` | `-` |  `outputs/2023-05-22/08-49-23`  |
| Framework | `sigmoid` | `softmax`   |  `false`   |     `false`   | `0.969136`  | `0.636225`  | `0.437534` | `0.408124` | `B` | `D` | `outputs/2023-05-22/08-18-17` |  
| Framework | `gumbel` |  `softmax`   |  `false`   |    `false`  |   `0.848765`  |  `0.763983`    | `0.651922` | `0.611969` |   `B` |  `C`   |  `outputs/2023-05-22/08-04-48`  |  
| Framework | `gumbel` |  `entmax`   |  `false`   |    `false`  |   `0.842593`  |  `0.748309`     | `0.742202` | `0.736384` |  `A`  |  `B`  |  `outputs/2023-05-22/09-13-40`  | 
| Framework | `gumbel` |  `softmax`   |  `true`   |    `false`  |   `0.731482`  |  `0.707190`     | `0.670618` | `0.637436` |   `A` |  `B`  |  `outputs/2023-05-22/09-38-41`  | 
| Framework | `gumbel` |  `entmax`   |  `true`   |    `false`  |   `0.586420`  |  `0.691018`     | `0.582086` | `0.564636` |  `A`  |  `B`  |  `outputs/2023-05-22/11-03-11`  | 
| Framework | `gumbel` |  `softmax`   |  `false`   |    `true`  |   `0.814815`  |  `0.726690`    | `0.708535` | `0.673539` |  `A` | `D` |  `outputs/2023-05-22/09-53-54`  | 

### 2. CUB-200 

| model | activation | norm_fn | slot_norm | reg_dist | f1-score | purity | disentanglement | completeness | directory |
|:-----------|:----:|:----:|:----:|:----:|:----:|:-------:|:-------:|:-------:|:-----------|
| Baseline | `sigmoid` |   `-`   |  `-`   |   `-`   | `0.805452` | `0.573071` | `0.189394`| `0.196021` | `outputs/2023-05-26/07-31-18` |
| Baseline | `gumbel (0.01)` |   `-`   |   `-`   |  `-`   | `0.765`  | `0.578` | `0.193078` | `0.201281` | `outputs/2023-05-28/09-21-34`  |
| Framework | `gumbel (0.5)` |  `entmax`   |  `false`   |    `false`  |   `0.773003`  |  `0.593836`    | `0.229795` | `0.255646` |   `outputs/2023-05-27/10-34-06`  | 
| Framework | `gumbel (0.01)` |  `entmax`   |  `false`   |    `false`  |   `0.726`  |  `0.657`    | `X` | `X` |   `outputs/2023-05-27/19-41-06`  | 